In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot as plt

# Creating the dataset

To create a linear regression dataset, we need to create a bunch of random points along a line. The way we'll do this is by:
1. Using `numpy` to sample a bunch of random x values.
1. Map the random x values to y values on a line.
1. Add noise to the y values so they don't sit perfectly on the line.

In [ ]:
# The weights for the line we're going sample from
true_weights = {'m': 1, 'b': 1.5}

# These are the lower and upper bounds of the x values numpy will sample
min_x_range = 0
max_x_range = 10

# The number of points in our dataset.
num_samples = 50

# How much jitter we want to add onto the y values.
noise_factor = 1

In [ ]:
# Randomly sample a bunch of x values between min_x_range and max_x_range.
x_val = np.random.uniform(min_x_range, max_x_range, num_samples)

# Map the random x values to y values on the line.
y_line = true_weights['m'] * x_val + true_weights['b']

# Add noise to the y values so they don't sit perfectly on the line.
y_val = y_line + np.random.uniform(-noise_factor, noise_factor, num_samples)

# Let's store our dataset in an easily accesible form (called a pandas DataFrame)
dataset = pd.DataFrame({'x': x_val, 'y': y_val})

In [ ]:
# Let's view at our dataset!
fig = px.scatter(x=dataset['x'], y=dataset['y'])
fig.update_layout(
    xaxis_title="X values",
    yaxis_title="Y values")
fig.show()

# Defining the loss function

In [ ]:
# These are some functions that we're about to use in a second
def poly_coeffs(x, coeffs):
    order = len(coeffs)
    y = 0
    for i in range(order):
        y += coeffs[i]*x**i
    return y

def draw_L_by_m(dataset, weights):
    """
    Draws the loss as a function of weight m
    
    Args:
        dataset: the dataset of x, y pairs
        weights: the optimal m and b weights
    """
    a, b, c = 0, 0, 0
    for _, point in dataset.iterrows():
        a += point['x']**2
        b += -2 * point['x'] * point['y'] + 2 * weights['b'] * point['x']
        c += -2 * weights['b'] * point['y'] + weights['b']**2 + point['y']**2
    a /= len(dataset)
    b /= len(dataset)
    c /= len(dataset)
    
    x = np.linspace(-4, 6, 100)
    plt.plot(x, poly_coeffs(x, [c, b, a]))
    plt.ylabel("Value of Loss")
    plt.xlabel("Value of m (slope)")
    plt.show()
    
def draw_L_by_b(dataset, weights):
    """
    Draws the loss as a function of weight b
    
    Args:
        dataset: the dataset of x, y pairs
        weights: the optimal m and b weights
    """
    a, b, c = 0, 0, 0
    print(weights)
    for _, point in dataset.iterrows():
        a += 1
        b += 2 * weights['m'] * point['x'] - 2 * point['y']
        c += point['x']**2 * weights['m']**2 \
             - 2 * weights['m'] * point['x'] * point['y'] + point['y']**2
    a /= len(dataset)
    b /= len(dataset)
    c /= len(dataset)
    x = np.linspace(-3.5, 6.5, 100)
    plt.plot(x, poly_coeffs(x, [c, b, a]))
    plt.ylabel("Value of Loss")
    plt.xlabel("Value of b (y-intercept)")
    plt.show()

In [ ]:
# Here's our loss as a function of m
# You could try and take the derivative of this polynomial, but there's a problem...
# We're drawing this polynomials with the true weights. We don't actually know them...
draw_L_by_m(dataset, true_weights)

In [ ]:
# Here's our loss as a function of b
draw_L_by_b(dataset, true_weights)

# Learning the model

In [ ]:
# These are some functions that we're going to use to help learn the model
def draw_plot_and_model(dataset, weights):
    """
    Draws the dataset and the model on a plot.
    
    Args:
        dataset: the dataset of x, y pairs
        weights: these are the weights of the model you want drawn. It should have an m and b.
        
    Returns:
        Plotly figure with visualized dataset and model
    """
    # This will be used to draw current model's regression line
    line_df = pd.DataFrame(dict(
        x = [dataset['x'].min(), dataset['x'].max()],
        y = [weights['m'] * dataset['x'].min() + weights['b'], 
             weights['m'] * dataset['x'].max() + weights['b']]
    ))
    # Draw our model's regression line
    fig_line = px.line(line_df, x="x", y="y")
    fig_line.update_traces(line_color='red', line_width=4)

    # Draw our dataset's points
    fig_scat = px.scatter(x=dataset['x'], y=dataset['y'])

    # Draw the points and regression line together
    fig_final = go.Figure(data=fig_line.data + fig_scat.data)
    fig_final.update_layout(
        xaxis_title="X values",
        yaxis_title="Y values")
    
    return fig_final

def dL_dm(weights, point):
    """
    Returns the value of the derivative of the loss with respect
    to m at given points and weights.
    
    Args:
        weights: m, b weights to calculate loss derivative at.
        point: x, y point to calculate loss derivative at.
        
    Returns:
        Value of dL/dm at point and weights.

    """
    # We did the math in class:
    # dL/dm = -2x * (y - mx - b)
    return -2 * point['x'] * (point['y'] - weights['m'] * point['x'] - weights['b'])

def dL_db(weights, point):
    """
    Returns the value of the derivative of the loss with respect
    to b at given points and weights.
    
    Args:
        weights: m, b weights to calculate loss derivative at.
        point: x, y point to calculate loss derivative at.
        
    Returns:
        Value of dL/db at point and weights.
    """
    # We did the math in class:
    # dL/db = -2 * (y - mx - b)
    return -2 * (point['y'] - weights['m'] * point['x'] - weights['b'])

def take_step(weights, point):
    """
    With the current weights and point, calculate the new weights 
    in the direction of the optimal weights.
    
    Args:
        weights: m, b weights to calculate loss derivative at
        point: x, y point to calculate loss derivative at
        
    Returns:
        The new weights
    """
    # Our "learning rate". This is what makes us take baby steps.
    # Fun activity: What happens when you make this bigger? What about smaller? Is it possible to be too big??
    lr = 1e-3

    # Update the weights m and b
    weights['m'] = weights['m'] - lr*dL_dm(weights, point)
    weights['b'] = weights['b'] - lr*dL_db(weights, point)
    return weights


## Initialization

In [ ]:

# Pick our random weights
# Note: these aren't really random right now because I want a visually appealing initial model
weights = {'m': -0.25,'b': 5}

# Let's draw our model and see the initial model with our data 
draw_plot_and_model(dataset, weights)

## First step

In [ ]:
# Let's sample a random point
rand_idx = np.random.randint(len(dataset))
point = dataset.iloc[rand_idx]

# And take a step calculating our new weights
new_weights = take_step(weights, point)

# Set our old weights to our new weights
weights = new_weights

fig = draw_plot_and_model(dataset, weights)
fig.show()

## Second step

In [ ]:
# Let's sample another random point
rand_idx = np.random.randint(len(dataset))
point = dataset.iloc[rand_idx]

# And take another step calculating our new weights
new_weights = take_step(weights, point)

# Set our old weights to our new weights
weights = new_weights

fig = draw_plot_and_model(dataset, weights)
fig.show()

## Tenth step

In [ ]:
# Now we do this 10x in a row
# Does the plot look any better?
for _ in range(2, 10):
    # Let's sample a random point
    rand_idx = np.random.randint(len(dataset))
    point = dataset.iloc[rand_idx]

    # And take a step calculating our new weights
    new_weights = take_step(weights, point)

    # Set our old weights to our new weights
    weights = new_weights

fig = draw_plot_and_model(dataset, weights)
fig.show()

## Five thousandth step

In [ ]:
# How about 5000x in a row
# Does the plot look any better?
for _ in range(10, 5000):
    # Let's sample a random point
    rand_idx = np.random.randint(len(dataset))
    point = dataset.iloc[rand_idx]

    # And take a step calculating our new weights
    new_weights = take_step(weights, point)

    # Set our old weights to our new weights
    weights = new_weights

fig = draw_plot_and_model(dataset, weights)
fig.show()

# Fun challenges for yourself

* Try changing the value of `lr`. How does that affect the number of steps you need to get a good looking model?
* Try a different loss function: absolute value, log-cosh error, huber loss
* Can you change this to estimate a polynomial instead of a line?